In [10]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from lab_utils_multiclass_TF import *
import logging
np.set_printoptions(precision=2)
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [11]:
#4 class dtaset for classification
classes = 4
m = 100
centers = [[-5,2],
           [-2,2],
           [1,2],
           [5,-2]]
std = 1.0
#making the datasets 
x_train, y_train = make_blobs(n_samples = m, centers = centers, cluster_std = std, random_state = 30)



In [12]:
plt_mc(x_train, y_train, classes, centers, std = std)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
#classes in data set
print(f'Unique Classes {np.unique(y_train)}')

#how classes are represented
print(f'Class Representation {y_train[:10]}')

#shapes of dataset
print(f'shape of x_train {x_train.shape}')

print(f'shape of y_train {y_train.shape}')


Unique Classes [0 1 2 3]
Class Representation [3 3 3 0 3 3 3 3 2 0]
shape of x_train (100, 2)
shape of y_train (100,)


Unlike the binary classification networks, this network has four outputs, one for each class. Given an input example, the output with the highest value is the predicted class of the input.   

Below is an example of how to construct this network in Tensorflow. Notice the output layer uses a `linear` rather than a `softmax` activation. While it is possible to include the softmax in the output layer, it is more numerically stable if linear outputs are passed to the loss function during training. If the model is used to predict probabilities, the softmax can be applied at that point.

In [14]:
tf.random.set_seed(1234)
model = Sequential(
    [
        Dense(2, activation = 'relu', name = 'L1'),
        Dense(4, activation = 'linear', name = 'L2')
    ]
)

The statements below compile and train the network. Setting `from_logits=True` as an argument to the loss function specifies that the output activation was linear rather than a softmax.

In [15]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

In [16]:
model.fit(
    x_train, y_train,
    epochs = 200
)

Epoch 1/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3913
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3474
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3161
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2832
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 1.2526
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 1.2215
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1902
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1538
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1175
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0730
Epoch 11/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0322
Epoch 12/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9871
Epoch 13/200
4/4 [==============================] - 0s 1ms/st

In [19]:
#With the model trained, we can see how the model has classified the training data.
plt_cat_mc(x_train, y_train, model, classes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Below, we will pull the trained weights from the model and use that to plot the function of each of the network units. Further down, there is a more detailed explanation of the results. You don't need to know these details to successfully use neural networks, but it may be helpful to gain more intuition about how the layers combine to solve a classification problem.

In [22]:
L1 = model.get_layer('L1')
W1,b1 = L1.get_weights()

In [24]:
# plot the function of the first layer
plt_layer_relu(x_train, y_train.reshape(-1,), W1, b1, classes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
# gather the trained parameters from the output layer
l2 = model.get_layer("L2")
W2, b2 = l2.get_weights()
# create the 'new features', the training examples after L1 transformation
Xl2 = np.maximum(0, np.dot(x_train,W1) + b1)

plt_output_layer_linear(Xl2, y_train.reshape(-1,), W2, b2, classes,
                        x0_rng = (-0.25,np.amax(Xl2[:,0])), x1_rng = (-0.25,np.amax(Xl2[:,1])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …